<a href="https://colab.research.google.com/github/sunghyunjun/kaggle-otto-recsys/blob/main/otto_transformers4rec_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check colab gcp, gpu environment

In [ ]:
# Check colab's gcp region
!curl ipinfo.io

{
  "ip": "34.135.132.227",
  "hostname": "227.132.135.34.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS396982 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
gcs_bucket = "sunghyun-kaggle-otto-us"
# gcs_bucket = "sunghyun-kaggle-otto-eu"
# gcs_bucket = "sunghyun-kaggle-otto-asia"

In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-3481e7d1-5be8-8eae-75f3-f8969d54ae9c)


In [ ]:
import multiprocessing
print(multiprocessing.cpu_count()) # Count the number of cores in a computer

12


# GCP personal setting for gcs, kaggle, neptune.ai

In [ ]:
# Authenticate to GCP
from google.colab import auth
auth.authenticate_user()

In [ ]:
# GCP secret manager, after run pip, notebook should be restarted
# !pip3 install --user google-cloud-secret-manager==2.5.0
!pip3 install --user google-cloud-secret-manager
from google.cloud import secretmanager


def get_secret(project_id, secret_name):
    """
    secret_name => To be replaced with your secret name
    project_id  => To be replaced with your GCP Project
    """

    # Create a Client:
    client = secretmanager.SecretManagerServiceClient()
    # secret_name = "kaggle" # => To be replaced with your secret name
    # project_id = 'otto-recsys' # => To be replaced with your GCP Project

    # Forge the path to the latest version of your secret with an F-string:
    resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest" 

    # Get your secret :
    response = client.access_secret_version(request={"name": resource_name})
    secret_string = response.payload.data.decode('UTF-8')

    # Tada ! you secret is in the secret_string variable!
    return secret_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Install gcsfuse

In [ ]:
!pip install gcsfs --upgrade -q
!pip install psutil --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 6.3 MB/s eta 0:00:00


In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  52608      0 --:--:-- --:--:-- --:--:-- 52608
OK
65 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 129499 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.12_amd64.deb ...
Unpacking gcsfuse (0.41.12) ...
Setting up gcsfuse (0.41.12) ...


In [ ]:
!mkdir ./mount_checkpoint
!gcsfuse {gcs_bucket} ./mount_checkpoint

2023/01/31 10:14:33.872360 Start gcsfuse/0.41.12 (Go version go1.18.4) for app "" using mount point: /content/mount_checkpoint
2023/01/31 10:14:33.884889 Opening GCS connection...
2023/01/31 10:14:34.101737 Mounting file system "sunghyun-kaggle-otto-us"...
2023/01/31 10:14:34.102421 File system has been successfully mounted.


# Kaggle API credentials

In [ ]:
with open('kaggle.json', 'a') as fp:
    fp.write(get_secret("otto-recsys", "kaggle"))

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


# Download Kaggle dataset

In [ ]:
!kaggle kernels output sunghyunjun/otto-etl-nvtabular-cpu-candidates-orders -p dataset


Output file downloaded to dataset/candidates_orders.npy
Output file downloaded to dataset/test/_file_list.txt
Output file downloaded to dataset/test/_metadata
Output file downloaded to dataset/test/_metadata.json
Output file downloaded to dataset/test/part_0.parquet
Output file downloaded to dataset/test/part_1.parquet
Output file downloaded to dataset/test/part_2.parquet
Output file downloaded to dataset/test/part_3.parquet
Output file downloaded to dataset/test/part_4.parquet
Output file downloaded to dataset/test/part_5.parquet
Output file downloaded to dataset/test/part_6.parquet
Output file downloaded to dataset/test/part_7.parquet
Output file downloaded to dataset/test/part_8.parquet
Output file downloaded to dataset/test/schema.pbtxt
Output file downloaded to dataset/train/_file_list.txt
Output file downloaded to dataset/train/_metadata
Output file downloaded to dataset/train/_metadata.json
Output file downloaded to dataset/train/part_0.parquet
Output file downloaded to dataset/

# Install Transformers4Rec

In [ ]:
!pip install transformers4rec[pytorch,nvtabular]
!pip install -U nvtabular==1.3.3
!pip install -U pytorch_lightning==1.8.0.post1
!pip install torchmetrics==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.3/774.3 KB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prep

# Import Transformers4Rec

In [ ]:
import gc
import glob
import os
import warnings

import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

from merlin_standard_lib import Schema, Tag
from transformers4rec import torch as tr
from transformers4rec.torch import Trainer
from transformers4rec.torch.ranking_metric import RecallAt
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch.utils.data_utils import T4RecDataLoader

# Variables Setting

In [ ]:
DEBUG = False
OUTPUT_PATH = "orders"
CHECKPOINT = "orders/checkpoint-50000"

In [ ]:
MAX_SEQUENCE_LENGTH = 500
D_MODEL = 512

TRAIN_BATCH_SIZE = 192
EVAL_BATCH_SIZE = 256


if DEBUG:
    MAX_STEPS = 2000
    EVAL_BATCH_SIZE = 4
else:
    MAX_STEPS = -1

In [ ]:
if DEBUG:
    train_path = [
        "./dataset/train/part_0.parquet",
        "./dataset/train/part_1.parquet",
        "./dataset/train/part_2.parquet",
        "./dataset/train/part_3.parquet",
    ]
    valid_path = "./dataset/valid/part_0.parquet"
    test_path = "./dataset/test/part_0.parquet"
else:
    train_path = sorted(glob.glob("./dataset/train/part_*.parquet"))
    valid_path = sorted(glob.glob("./dataset/valid/part_*.parquet"))
    test_path = sorted(glob.glob("./dataset/test/part_*.parquet"))
    
output_path = os.path.join("mount_checkpoint", OUTPUT_PATH)
checkpoint = os.path.join("mount_checkpoint", CHECKPOINT)

# Create schema proto manually

(on Oct 6, 2021)
> The latest version of the NVTabular library (used for the ETL pipeline) automatically outputs a schema file in the protobuf text format (schema.pbtxt) together with the parquet files of processed data. This is still a work in progress and we'll include the necessary support in the future release of Transformers4Rec.

https://github.com/NVIDIA-Merlin/Transformers4Rec/issues/271#issuecomment-934850026

In [ ]:
%%writefile schema.pb
feature {
  name: "session"
  type: INT
  int_domain {
    name: "session"
    min: 0
    max: 12899778
    is_categorical: true
  }
  annotation {
    tag: "context"
    tag: "categorical"
  }
}

feature {
  name: "aid-list"
  value_count {
    min: 1
    max: 500
  }
  type: INT
  int_domain {
    name: "aid-list"
    min: 1
    max: 657941
    is_categorical: true
  }
  annotation {
    tag: "item"
    tag: "categorical"
    tag: "list"
    tag: "item_id"
  }
}

Overwriting schema.pb


In [ ]:
schema = Schema().from_proto_text("schema.pb")

In [ ]:
schema_test = schema.select_by_name(
    [
        "aid-list",
    ]
)

# Build Model blocks

In [ ]:
inputs = tr.TabularSequenceFeatures.from_schema(
    schema_test,
    embedding_dims={"aid-list": D_MODEL},
    max_sequence_length=MAX_SEQUENCE_LENGTH,
    d_output=D_MODEL,
    masking="clm",
)

In [ ]:
inputs

TabularSequenceFeatures(
  (to_merge): ModuleDict(
    (categorical_module): SequenceEmbeddingFeatures(
      (filter_features): FilterFeatures()
      (embedding_tables): ModuleDict(
        (aid-list): Embedding(657942, 512, padding_idx=0)
      )
    )
  )
  (_aggregation): ConcatFeatures()
  (projection_module): SequentialBlock(
    (0): DenseBlock(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (_masking): CausalLanguageModeling()
)

In [ ]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=D_MODEL, n_head=4, n_layer=2, total_seq_length=MAX_SEQUENCE_LENGTH
)

# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs,
    tr.MLPBlock([D_MODEL]),
    tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

In [ ]:
# Defines the evaluation top-N metrics and the cut-offs
metrics = [
    RecallAt(top_ks=[10, 20], labels_onehot=True)
]

# Define a head related to next item prediction task 
# Important: transformerc4rec v0.1.15
# NextItemPredictionTask should have parameter [hf_format=True]
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, hf_format=True, metrics=metrics),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = tr.Model(head)

# Configure T4RecTrainingArguments and Trainer

In [ ]:
# Set hyperparameters for training 
train_args = T4RecTrainingArguments(
    data_loader_engine='nvtabular', 
    dataloader_drop_last=True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=TRAIN_BATCH_SIZE, 
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    output_dir=output_path, 
    learning_rate=0.0005,
    lr_scheduler_type='cosine', 
    learning_rate_num_cosine_cycles_by_epoch=1,
    num_train_epochs=1,
    max_sequence_length=MAX_SEQUENCE_LENGTH, 
    logging_steps=500,
    save_steps=1000,
    save_total_limit=5,
    no_cuda=False,
    fp16=True,
    dataloader_num_workers=4,
    max_steps=MAX_STEPS,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema_test,
    compute_metrics=True,
)

Using amp half precision backend


# Prepare Evaluation

In [ ]:
trainer.load_model_trainer_states_from_checkpoint(checkpoint, model=model)

In [ ]:
model.eval()

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (aid-list): Embedding(657942, 512, padding_idx=0)
              )
            )
          )
          (_aggregation): ConcatFeatures()
          (projection_module): SequentialBlock(
            (0): DenseBlock(
              (0): Linear(in_features=512, out_features=512, bias=True)
              (1): ReLU(inplace=True)
            )
          )
          (_masking): CausalLanguageModeling()
        )
        (1): SequentialBlock(
          (0): DenseBlock(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): ReLU(inplace=True)
          )
        )
        (2): TansformerBlock(
          (transformer): XLNetModel(
            (word_embe

In [ ]:
candidates = np.load("./dataset/candidates_orders.npy")

In [ ]:
encoded_to_aid = dict(enumerate(candidates, start=2))
aid_to_encoded = {v: k for k, v in encoded_to_aid.items()}

In [ ]:
def to_encode_(aids):
    return [aid_to_encoded.get(x, 1) for x in aids]

def to_encode(col):
    result = []
    for aids in col:
        result.append(to_encode_(aids))
    return result

In [ ]:
def aid_to_decode(aid, start_index=2):
    # index 0: padding, index 1: OOV, index 2: start index
    if aid >= start_index:
        result = candidates[aid - start_index]
    # OOV to candidates[0]
    elif aid == 1:
        result = candidates[0]
    else:
        result = 0
        
    return result

In [ ]:
def to_decode_(aids, start_index=2):
    # index 0: padding, index 1: OOV, index 2: start index
    result = []
    for e in np.nditer(aids):
        if e >= start_index:
            decoded = candidates[e - start_index]
        # OOV to candidates[0]
        elif e == 1:
            decoded = candidates[0]
        else:
            decoded = 0
            
        result.append(decoded)
    return np.asarray(result)

# Prepare DataLoader for validation

In [ ]:
def zero_padding(x, length):
    x += [0] * (length - len(x))
    return x

In [ ]:
from enum import Enum
class LabelTypes(Enum):
    CLICKS = 1
    CARTS = 2
    ORDERS = 3

In [ ]:
TOP_K = 20
LABEL_TYPES = ["clicks", "carts", "orders"]

In [ ]:
for k in LabelTypes:
    print(k.value)

1
2
3


In [ ]:
eval_dataload = T4RecDataLoader.parse('pyarrow').from_schema(
    schema,
    test_path,
    EVAL_BATCH_SIZE,
    max_sequence_length=MAX_SEQUENCE_LENGTH,
    drop_last=False,
    shuffle=False,
)

In [ ]:
df = pd.DataFrame(columns=["session_type", "labels"])

In [ ]:
# batch = next(iter(eval_dataload))

In [ ]:
# batch

# Run Valid

In [ ]:
for i_batch, batch in enumerate(tqdm(eval_dataload)):
    for k, v in batch.items():
        batch[k] = v.to("cuda:0")
        
    output = model(batch, training=False)

    sessions = batch["session"].detach().cpu().numpy()
    predictions = output["predictions"].detach().cpu().numpy()
    top_preds_ = np.argpartition(predictions, -TOP_K, axis=1)[:, -TOP_K:]
    top_preds = [to_decode_(x) for x in top_preds_]
    # numpy ndarray element to string elements with white space
    top_preds = [(" ").join(map(str, x.tolist())) for x in top_preds]

    for label_type in LabelTypes:
        session_types = [str(s) + "_" + label_type.name.lower() for s in sessions]
        
        df = pd.concat(
            [
                df,
                pd.DataFrame(
                    {
                        "session_type": session_types,
                        "labels": top_preds,
                    }
                )
            ], axis=0
        )
    
    if DEBUG:
        break
        

  0%|          | 0/6531 [00:00<?, ?it/s]

In [ ]:
df

,session_type,labels
0,12899779_clicks,1304914 927087 819288 1464360 209710 597742 27...
1,12899780_clicks,1586171 1263108 760500 1269469 455423 595830 1...
2,12899781_clicks,1307153 1515266 1102089 1228668 1681537 811448...
3,12899782_clicks,1443394 1169987 53600 1577479 363001 1694535 1...
4,12899783_clicks,1451449 1464627 351665 168330 1727436 1249267 ...
...,...,...
118,14571577_orders,1304914 927087 819288 1464360 209710 597742 27...
119,14571578_orders,1304914 927087 819288 1464360 209710 597742 27...
120,14571579_orders,843372 1280299 29173 1822490 1525131 1478374 1...
121,14571580_orders,1256802 765284 1537463 826148 180283 576317 17...


In [ ]:
df.to_csv(os.path.join("mount_checkpoint", OUTPUT_PATH, "submission.csv"), index=False)